In [1]:
import cv2,numpy as np
from keras.models import Sequential,Model #Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
#from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
import pandas as pd
image_size = 227

In [2]:
# Create AlexNet
model = keras.Sequential()
model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(image_size, image_size, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', 
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),     #使用SGD來
            metrics=['accuracy'])
#model.summary()

In [3]:
train_images,train_labels = [],[]
height = pd.read_csv('train.csv')
for index in range(len(height.index)):
#     print(index)     
    img = cv2.resize(cv2.imread('train_images/%s'%(str(height.iloc[index, 0])),0),(image_size,image_size))
    train_images.append(img)
    train_labels.append(height.iloc[index,1])
np.reshape(train_labels,(len(train_images),1))

array([[0],
       [1],
       [1],
       ...,
       [3],
       [0],
       [5]], dtype=int64)

In [4]:
from sklearn.model_selection import train_test_split
train_images = np.array(train_images,dtype=np.float32)
train_labels = np.array(train_labels,dtype=np.float32)
# print(train_images)
# print(train_images/255.0)
print(train_images.shape)
print(train_labels.shape)

(2528, 227, 227)
(2528,)


In [5]:
import tensorflow as tf
with tf.device('/gpu:0'):
    history = model.fit(train_images,train_labels,epochs=110,batch_size=23,validation_split=0.7) # batch_size=23  epochs=110 or batch_size=110  epochs=23

Epoch 1/110
33/33 [==============================] - 37s 1s/step - loss: 1.6814 - accuracy: 0.5844 - val_loss: 11.5063 - val_accuracy: 0.1932
Epoch 2/110
33/33 [==============================] - 29s 891ms/step - loss: 0.7415 - accuracy: 0.7546 - val_loss: 5.8041 - val_accuracy: 0.2593
Epoch 3/110
33/33 [==============================] - 29s 881ms/step - loss: 0.4675 - accuracy: 0.8536 - val_loss: 6.9280 - val_accuracy: 0.2576
Epoch 4/110
33/33 [==============================] - 31s 959ms/step - loss: 0.3158 - accuracy: 0.8905 - val_loss: 3.6536 - val_accuracy: 0.3040
Epoch 5/110
33/33 [==============================] - 31s 945ms/step - loss: 0.3591 - accuracy: 0.8747 - val_loss: 2.2931 - val_accuracy: 0.4062
Epoch 6/110
33/33 [==============================] - 33s 996ms/step - loss: 0.2555 - accuracy: 0.9208 - val_loss: 1.4231 - val_accuracy: 0.4910
Epoch 7/110
33/33 [==============================] - 33s 992ms/step - loss: 0.1689 - accuracy: 0.9314 - val_loss: 1.3038 - val_accuracy: 0

In [6]:
import random,numpy as np
x = random.randint(0, len(train_images))
result = model.predict(train_images[x].reshape(-1,image_size,image_size,1))
print(result)
print(x,np.argmax(result))

1/1 [==============================] - 0s 133ms/step
[[1.4983402e-09 2.4675426e-06 2.9470803e-08 9.9999702e-01 3.3027757e-07
  2.0456577e-07]]
2228 3


In [7]:
with tf.device('/gpu:0'):
    train_loss, train_acc = model.evaluate(train_images,  train_labels)
print('test_loss:', train_loss)
print('test_acc:', train_acc)

79/79 [==============================] - 15s 192ms/step - loss: 0.0557 - accuracy: 0.9822
test_loss: 0.055721625685691833
test_acc: 0.9821993708610535


In [8]:
model.save('Final_AOI_Alexnet_%d.h5'%(image_size))

In [9]:
from keras.models import load_model
import csv,pandas as pd,cv2,numpy as np
model = load_model('Final_AOI_Alexnet_%d.h5'%(image_size)) #載入訓練好的模型
# Load Test Data
test_images = []
labels = []
height = pd.read_csv('test.csv')
with tf.device('/gpu:1'):
    for index in range(len(height.index)):    
        img = cv2.resize(cv2.imread('test_images/%s'%(str(height.iloc[index, 0])),-1),(image_size,image_size))
        test_images.append(str(height.iloc[index, 0]))
        result = model.predict(img.reshape(-1,image_size,image_size,1))
        labels.append(np.argmax(result))

with open('testalex.csv','w',newline='') as file2:
    filename = ['ID','Label']
    write = csv.DictWriter(file2,fieldnames=filename)
    write.writeheader()
    for i,j in enumerate(labels):
        if i%1000 == 0:
            print(i,j)
        write.writerow({"ID": test_images[i],'Label': j})
                


1/1 [==============================] - 0s 32ms/step
0 1
1000 0
2000 0
3000 1
4000 3
5000 1
6000 0
7000 5
8000 5
9000 0
10000 5
